In [1]:
import os
import re
import ast
import json
import string
import openai
import psycopg2
import collections
import numpy as np
import pandas as pd
import pandas_dedupe
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from neo4j import GraphDatabase
from jsonpath_ng.ext import parse
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [2]:
ps = PorterStemmer()
connection = psycopg2.connect(
        dbname='nourish',
        user='v1desai@ucsd.edu',
        password='emfQGcx3',
        host='awesome-hw.sdsc.edu',
        port=5432
    )
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"

In [3]:
with open('ingredient_and_instructions.json') as file:
    tasty_json_ingredients = json.load(file)
ingredients_and_recipes_df = pd.DataFrame(tasty_json_ingredients)

In [4]:
#load in tasty dishes (csv)
df_tasty_dishes = pd.read_csv('dishes.csv',usecols=['slug', 'protein', 'fat', 'calories', 'sugar', 'carbohydrates', 'fiber']) 
df_tasty_dishes.head()

,slug,protein,fat,calories,sugar,carbohydrates,fiber
0,homemade-cinnamon-rolls,7.0,21.0,479.0,24.0,63.0,1.0
1,whipped-coffee,0.0,0.0,69.0,18.0,18.0,0.0
2,fluffy-perfect-pancakes,36.0,50.0,1102.0,12.0,123.0,3.0
3,tasty-101-cinnamon-rolls,8.0,25.0,562.0,28.0,74.0,1.0
4,healthy-banana-pancakes,7.0,4.0,184.0,9.0,30.0,4.0


In [5]:
#unstructured data
jsonpath_expr = parse("$..['instructions']")
matches = [match.value for match in jsonpath_expr.find(tasty_json_ingredients)]

text = []
for i in range(len(matches)):
    list_text = " ".join(line['display_text'].strip() for line in matches[i])
    text.append(list_text)

In [6]:
#extract all products
expr = parse("$")
list_products = [match.value.keys() for match in expr.find(tasty_json_ingredients)]

In [7]:
jsonpath_expr = parse("$..['ingredient_sections']")
matches = [match.value for match in jsonpath_expr.find(tasty_json_ingredients)]

In [8]:
#extracting list of ingredients product by product
ingredients = []
ingredient_per_prod = []
for products in matches:
    for product in products:
        ing_per_prod = product['ingredients']
        for ing in ing_per_prod:
            ingredient_per_prod.append(ing['name'])
    ingredients.append(ingredient_per_prod)
    ingredient_per_prod = []

In [9]:
# extract all metric_unit for each ingredient per product
metric = []
metric_per_prod = []
for products in matches:
    for product in products:
        met_per_prod = product['ingredients']
        for met in met_per_prod:
            if met['metric_unit'] is not None:
                metric_per_prod.append((met['metric_unit']['quantity'],met['metric_unit']['display']))
            else:
                metric_per_prod.append((None,None))
    metric.append(metric_per_prod)
    metric_per_prod = []

In [10]:
col_products = pd.Series(list_products[0])
col_ingredients = pd.Series(ingredients)
col_metric_unit = pd.Series(metric)
col_instructions = pd.Series(text)

In [11]:
df = pd.DataFrame()
df['products'] = col_products
df['ingredients'] = col_ingredients
df['metric_unit'] = col_metric_unit
df['instructions'] = col_instructions
df.head()

,products,ingredients,metric_unit,instructions
0,homemade-cinnamon-rolls,"[unsalted butter, whole milk, granulated sugar...","[(115, g), (480, mL), (100, g), (None, None), ...",Generously butter two disposable foil pie/cake...
1,whipped-coffee,"[hot water, sugar, instant coffee powder, milk...","[(28, g), (24, g), (12, g), (None, None), (Non...","Add the hot water, sugar, and instant coffee t..."
2,fluffy-perfect-pancakes,"[flour, baking powder, milk, butter, egg yolks...","[(500, g), (None, None), (960, mL), (170, g), ...",Whisk together the flour and baking powder in ...
3,tasty-101-cinnamon-rolls,"[whole milk, sugar, unsalted butter, active dr...","[(480, mL), (100, g), (None, None), (None, Non...","Make the dough: In a large bowl, whisk togethe..."
4,healthy-banana-pancakes,"[ripe bananas, eggs, vanilla extract, quick-co...","[(None, None), (None, None), (None, None), (70...",Mash bananas in a large bowl until smooth. Mix...


In [13]:
df_products = df.merge(df_tasty_dishes,how='inner',left_on='products',right_on='slug')
df_products.drop('slug',axis=1,inplace=True)

In [15]:
df_products.dropna(inplace = True)
df_products.reset_index(inplace = True, drop = False)

In [17]:
df_final = pandas_dedupe.dedupe_dataframe(df_products,['products','ingredients'])

Importing data ...


C:\Users\varun\AppData\Local\Temp\ipykernel_22392\3765877714.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_final = pandas_dedupe.dedupe_dataframe(df_products,['products','ingredients'])


Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 4205


In [32]:
df_final['ingredients'] = df_final.apply(lambda row : [ingredient.strip()[1:-1] for ingredient in row['ingredients'][1:-1].split(', ')], axis =1)

In [33]:
def extract_keywords(products):
    keywords = [ps.stem(prod) for prod in products.split('-')]
    counter = collections.Counter(keywords)
    return [word for word,count in counter.most_common()[:2]]

def add_keywords(products_df):
    cluster_groups = df_final.groupby('cluster id')['products'].agg(list)
    cluster_keywords_df = pd.DataFrame(index=cluster_groups.index)
    cluster_keywords_df['cluster_keywords'] = cluster_groups.apply(lambda products_list: ', '.join(extract_keywords('-'.join(products_list))))
    
    # Merge the cluster_keywords back to the original dataframe
    df= pd.merge(df_final, cluster_keywords_df, on='cluster id')
    
    # Display the resulting dataframe
    df = df[['products','ingredients','metric_unit','instructions','calories','protein','fat','sugar','carbohydrates','fiber','cluster id','cluster_keywords']]
    return df

In [34]:
df = add_keywords(df_final)

In [35]:
def clean_text(text):
    text = re.sub('\W+',' ', str(text))
    return text.lower()
    
def get_experimental_table():
    cur = connection.cursor()
    exp_query = """SELECT fdc_id, description FROM usda_2022_food_branded_experimental"""
    cur.execute(exp_query)
    records = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    cur.close()
    
    df_experimental = pd.DataFrame(records, columns = columns)
    df_experimental.drop_duplicates(subset = 'description', keep = 'first', inplace = True)
    df_experimental.reset_index(drop = True, inplace = True)
    df_experimental['description'] = df_experimental.apply(lambda row :clean_text(row['description']), axis = 1)
    return df_experimental

In [36]:
df_experimental = get_experimental_table()

In [37]:
def add_usda_rows(df, df_experimental, cluster_id):
    clus_key = df.groupby('cluster id').get_group(cluster_id)['cluster_keywords']
    matches = process.extract(clus_key.values[0], df_experimental['description'],scorer=fuzz.token_sort_ratio)
    threshold = 95
    filtered_matches = [match for match in matches if match[1] >= threshold]
    fdc_to_match = []
    # Extract rows based on the filtered matches
    for match in filtered_matches:
        index = match[2]
        matched_row = df_experimental.iloc[index]
        fdc_to_match.append(matched_row.fdc_id)
    if len(fdc_to_match)>0:
        fdc_to_match = tuple(fdc_to_match)
        sql_query = f"""SELECT t1.fdc_id, t1.description, t2.amount, t3.name, t3.unit_name, t4.ingredients \
        FROM usda_2022_food_branded_experimental t1 \
        LEFT JOIN usda_2022_branded_food_nutrients t2 ON t1.fdc_id = t2.fdc_id \
        LEFT JOIN usda_2022_branded_food_product t4 ON t1.fdc_id = t4.fdc_id \
        LEFT JOIN usda_2022_nutrient_master t3 ON t2.nutrient_id = t3.id \
        WHERE t1.fdc_id IN {fdc_to_match}"""
        
        cur = connection.cursor()
        cur.execute(sql_query)
        records = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        cur.close()
        records_df = pd.DataFrame(records, columns = columns)
        for fdc in fdc_to_match:
            new_df = records_df.groupby('fdc_id').get_group(fdc)
            new_row_append = pd.Series([new_df['description'].values[0].lower(), \
                                re.sub("[\(\[].*?[\)\]]", "",new_df.ingredients.values[0].lower()), \
                                None, \
                                None, \
                               float(new_df[new_df['name'] == 'Energy']['amount'].values[0]), \
                               float(new_df[new_df['name'] == 'Protein']['amount'].values[0]), \
                               float(new_df[new_df['name'] == 'Total lipid (fat)']['amount'].values[0]), \
                               float(new_df[new_df['name'] == 'Sugars, Total']['amount'].values[0]), \
                               float(new_df[new_df['name'] == 'Carbohydrate, by difference']['amount'].values[0]), \
                               float(new_df[new_df['name'] == 'Fiber, total dietary']['amount'].values[0]), \
                               cluster_id, \
                               clus_key[0]], index=df.columns)
            df = pd.concat([df, pd.DataFrame([new_row_append], columns=df.columns)], ignore_index=True)
        return df
    else:
        print("No match found, records not inserted")
        return df

In [38]:
df = add_usda_rows(df, df_experimental, 4200)

No match found, records not inserted


In [45]:
def create_graph(tx, cluster_id, cluster_keywords, product_name, ingredients, instructions, protein, fat, calories, sugar, carbohydrates, fiber):
    # Create Product node
    tx.run("""
        MERGE (c:Cluster {name: $cluster_id, keywords: $cluster_keywords})
    """, cluster_id=cluster_id, cluster_keywords=cluster_keywords)

    tx.run("""
        MERGE (p:Product {name: $product_name, protein: $protein, fat: $fat,
                          calories: $calories, sugar: $sugar, carbohydrates: $carbohydrates, fiber: $fiber})
    """, product_name=product_name, protein=protein, fat=fat, calories=calories, sugar=sugar, carbohydrates=carbohydrates, fiber=fiber)
    
    # Create HAS_PRODUCTS relationships between cluster and Product
    if product_name:
        tx.run("""MATCH (c:Cluster {name: $cluster_id})
                  MATCH (p:Product {name: $product_name})
                  MERGE (c)-[:HAS_PRODUCTS]->(p)
               """, cluster_id=cluster_id, product_name=product_name)
            
    # Create Ingredient nodes
    for ingredient in ingredients:
        if ingredient:
            tx.run("""
                MERGE (i:Ingredient {name: $ingredient})
            """, ingredient=ingredient)

    # Create CONTAINS relationships between Product and Ingredient
    for ingredient in ingredients:
        if ingredient:
            tx.run("""
                MATCH (p:Product {name: $product_name})
                MATCH (i:Ingredient {name: $ingredient})
                MERGE (p)-[:CONTAINS]->(i)
            """, product_name=product_name, ingredient=ingredient) 

    # Create Instruction nodes
    if instructions:
        tx.run("""
            MERGE (j:Instructions {instructions: $instructions})
        """, instructions=instructions)
    #Create HAS_INSTRUCTIONS relationships between Product and Ingredient
        tx.run("""
                MATCH (p:Product {name: $product_name})
                MATCH (j:Instructions {instructions: $instructions})
                MERGE (p)-[:HAS_INSTRUCTIONS]->(j)
            """, product_name=product_name, instructions=instructions)
        
# Connect to the database and run the transaction
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        for index, row in df.iterrows():
            session.execute_write(create_graph,
                                  row['cluster id'],
                                  row['cluster_keywords'],
                                  row['products'],
                                  row['ingredients'],
                                  row['instructions'],
                                  row['protein'],
                                  row['fat'],
                                  row['calories'],
                                  row['sugar'],
                                  row['carbohydrates'],
                                  row['fiber'])